In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from Bio.Blast import NCBIXML
from Bio import Entrez, SeqIO
import requests
import re
from tqdm import tqdm
import json

In [ ]:
#Read in the uniprotKB proteins ; this file is from searching the uniprot website

with open('uniprotkb_Nicotine_AND_taxonomy_id_4085_2023_07_25.json') as f:
    data = json.load(f)

#get all the uniprot_ids
uniprot_ids_og  = []
for i in data['results']:
  uniprot_ids_og.append(i['primaryAccession'])

In [ ]:
#Read in the blast results ; this file is from running pBlast with the browser tool
with open('BY9H05C4016-Alignment.json') as f:
    blast_results = json.load(f)

In [ ]:
#blast_results['BlastOutput2'][0]['report']['results']['search']['hits'][0]['description'][0]['accession']
#len(blast_results['BlastOutput2'][x]['report']['results']['search']['hits'])
#base_protein_count = len(blast_results['BlastOutput2'])

'Q40479'

In [ ]:
# get the uniprotkb ids of the blast results
list_ids = []
for i in range(base_protein_count):
  total_hits = len(blast_results['BlastOutput2'][i]['report']['results']['search']['hits'])
  for x in range(total_hits):
    list_ids.append(blast_results['BlastOutput2'][i]['report']['results']['search']['hits'][x]['description'][0]['accession'])
print(len(list_ids))
print(len(set(list_ids)))

list_ids = list(set(list_ids))

27185
2566


In [ ]:
# filter out the uniprot results from blasted results
#Remove protein ids that are already in the uniprotkb set
uniprot_set = set(data)
list_result = [x for x in list_ids if x not in uniprot_ids_og]
len(list_result)

2525

In [ ]:
# sample ~600 proteins to include in the database
import random
random.seed(1)
sampled_blast = random.sample(list_result, 600)

In [ ]:

# Dictionary to hold all protein data
all_protein_data = {}
total = 0


for c, uniprot_id in enumerate(sampled_blast):
  try:
    response = requests.get(f"https://www.uniprot.org/uniprot/{uniprot_id}.json", timeout=10)
    print(c)
    if response.status_code == 200:
        protein_data = response.json()
        all_protein_data[uniprot_id] = protein_data
        total += 1
    else:
        print(f"Failed to retrieve data for {uniprot_id}. HTTP status code: {response.status_code}")
  except:
    print('timed out')
print(f' total timed out is {total}')

In [125]:
#combine the json for the uniprot results and the blast results
len(all_protein_data)

600

In [130]:
# store dataset in json for next part of pipeline
dataset_json = {}
dataset_json['blast_protein'] = all_protein_data
dataset_json['uniprot_protein'] = data


In [132]:
#save json
dataset_json_path = 'dataset_json.json'
with open(dataset_json_path, 'w') as f:
    json.dump(dataset_json, f)
